In [11]:
import numpy as np
import os
import json
from tqdm import tqdm
import argparse
#import nbimporter
from format_conversion import *

parser = argparse.ArgumentParser()

parser.add_argument('--json_path', default=r'annotations',
type=str,
help="input: coco format(json)")

parser.add_argument('--save_path', default=r'convert_results', type=str,
help="specify where to save the output dir of labels")
parser.add_argument('--json_type', default=r'xRayBone_train.json', type=str,
help="import train/val or test .json file?")
parser.add_argument('--save_type',default=r'train',type=str,
                    help="export train/val or test .txt file")
arg = parser.parse_args(args=[])

xiaoshu = 10 ** 6 # 保存几位小数, 6位


cwd = os.getcwd()
json_file = arg.json_path# Annotation of COCO Object Instance type
save_type = arg.save_type
save_file = arg.save_path # saved path
ana_txt_save_path = os.path.join(cwd,save_file)
json_type = arg.json_type
json_path = os.path.join(cwd, json_file)
file_path = os.path.join(json_path, json_type)

conv = my_converter(file_path,save_type,ana_txt_save_path)
conv.converter()


In [14]:
class my_converter:
    def __init__(self,file_path,save_type,ana_txt_save_path):
        self.file_path = file_path
        self.save_type = save_type
        self.ana_txt_save_path = ana_txt_save_path
    def converter(self):
        data = json.load(open(self.file_path, 'r'))
        if not os.path.exists(self.ana_txt_save_path):
            os.makedirs(self.ana_txt_save_path)

        id_map = {}  # The ids of the coco dataset are not continuous! Remap and output again!
        with open(os.path.join(self.ana_txt_save_path, 'classes.txt'), 'w') as f:
        # 写入classes.txt
            for i, category in enumerate(data['categories']):
            f.write(f"{category['name']}\n")
            id_map[category['id']] = i


        list_file = open(os.path.join(self.ana_txt_save_path, self.save_type+'.txt'), 'w')
        
        anno_count = 0 #count for same image annotations
        last_img_id = -1 #init img counting, make sure entering the loop
        keypoints = [] #init key points lists
        for annotations in tqdm(data['annotations']):
        # filename = annotations["file_name"]
            img_id = annotations["image_id"]
            if img_id != last_img_id:
                for images in data['images']:
                    if images["id"] == img_id:
                        img_width = images["width"]
                        img_height = images["height"]  
            if annotations["keypoints"] != None:
                keypoints = annotations["keypoints"]
                anno_count = 0 #reset anno count
            bbox = annotations["bbox"]
            bbox_center = self.bbox_converter(bbox)
            keypoints.append(bbox_center)    
            last_img_id = img_id
            anno_count += 1
            if anno_count == 3 : #gather all 3 annotations for one same img_id
                arry_x = np.zeros([4, 1])
                num_1 = 0
                for x in keypoints[0:12:3]:
                    arry_x[num_1, 0] = int((x / img_width) * xiaoshu) / xiaoshu
                    num_1 += 1

                arry_y = np.zeros([4, 1])
                num_2 = 0
                for y in keypoints[1:12:3]:
                    arry_y[num_2, 0] = int((y / img_height) * xiaoshu) / xiaoshu
                    num_2 += 1

                arry_v = np.zeros([4, 1])
                num_3 = 0
                for v in keypoints[2:12:3]:
                    arry_v[num_3, 0] = v
                    num_3 += 1
    
                    list_1 = []
                    num_4 = 0
                    for i in range(4):
                        list_1.append(float(arry_x[num_4]))
                        list_1.append(float(arry_y[num_4]))
                        list_1.append(float(arry_v[num_4]))
                    num_4 += 1


                fil = img_id

                cun_1 = os.path.join(self.ana_txt_save_path, str(fil))
                cun_2 = cun_1 + ".txt"

                with open(cun_2, "r") as f:
                    read_lines = f.readlines()
                x_1 = read_lines[0].strip().split(",")
                x_2 = x_1 + list_1
                list_xin = list(map(str, x_2))
                list_xin = " ".join(list_xin)
                with open(cun_2, "w") as f:
                    f.write(list_xin)

            for img in tqdm(data['images']):
                filename = img["file_name"]
                img_width = img["width"]
                img_height = img["height"]
                img_id = img["id"]
                head, tail = os.path.splitext(filename)
                ana_txt_name = head + ".txt"  # The corresponding txt name is consistent with jpg
                os.chdir(self.ana_txt_save_path)
                os.rename(str(img_id) + ".txt", ana_txt_name)

    def bbox_converter(self,bbox):
            x_min, y_min, bb_width, bb_height = bbox
            x_max = x_min + bb_width
            y_max = y_min + bb_height
            # compute center coordinates
            center_x = (x_min + x_max) / 2
            center_y = (y_min + y_max) / 2
            bbox_center = [center_x, center_y, 2]
            return bbox_center

  0%|                                                  | 0/2400 [00:00<?, ?it/s]


KeyError: 'width'